In [2]:
import torch
from gcn import GCN
from utils import load_data, preprocess, normalize_adj_tensor, accuracy, get_train_val_test
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from metattack import MetaApprox, Metattack
import seaborn as sns
import gc

In [3]:
# Fixed parameters for Cora dataset
seed = 15
epochs = 200
lr = 0.01
hidden = 64  # Changed from 16 to 64 as requested
dataset = 'cora'
model_variant = 'Meta-Self'  # Options: 'A-Meta-Self', 'Meta-Self'
ptb_rates = 0.05  # Multiple perturbation rates

In [4]:
# Set seeds for reproducibility
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != 'cpu':
    torch.cuda.manual_seed(seed)

# Load Cora dataset
adj, features, labels = load_data(dataset=dataset)
nclass = max(labels) + 1

Loading cora dataset...
reading cora...
Selecting 1 largest connected components


In [5]:
# Split dataset
val_size = 0.1
test_size = 0.8
train_size = 1 - test_size - val_size
idx = np.arange(adj.shape[0])
idx_train, idx_val, idx_test = get_train_val_test(idx, train_size, val_size, test_size, stratify=labels)
idx_unlabeled = np.union1d(idx_val, idx_test)

# Preprocess without normalizing adjacency yet
adj, features, labels = preprocess(adj, features, labels, preprocess_adj=False)

# Move data to device
if device != 'cpu':
    adj = adj.to(device)
    features = features.to(device)
    labels = labels.to(device)

In [6]:
def train_gcn(adj):
    ''' Train GCN on the given adjacency matrix '''
    # Normalize adjacency matrix for GCN
    norm_adj = normalize_adj_tensor(adj)
    
    # Initialize GCN model - the provided GCN is already 2-layer
    # We just need to set the hidden dimension to 64
    gcn = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=labels.max().item() + 1,
              dropout=0.5)
    
    if device != 'cpu':
        gcn = gcn.to(device)
    
    optimizer = optim.Adam(gcn.parameters(),
                           lr=lr, weight_decay=5e-4)
    
    # Train GCN
    gcn.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = gcn(features, norm_adj)
        loss_train = F.nll_loss(output[idx_train], labels[idx_train])
        loss_train.backward()
        optimizer.step()
    
    return gcn

In [7]:
def evaluate(gcn, adj, idx):
    ''' Evaluate GCN on the given adjacency matrix and index '''
    # Normalize adjacency matrix for evaluation
    norm_adj = normalize_adj_tensor(adj)
    
    gcn.eval()
    with torch.no_grad():
        output = gcn(features, norm_adj)
        loss = F.nll_loss(output[idx], labels[idx])
        acc = accuracy(output[idx], labels[idx])
    
    return acc.item()

In [8]:
def run_attack():
    results = {}
    
    print('=== Training GCN on original(clean) graph ===')
    # First, train the GCN on the clean graph
    trained_gcn = train_gcn(adj)
    
    # Set up attack model parameters
    if 'Self' in model_variant:
        lambda_ = 0
    if 'Train' in model_variant:
        lambda_ = 1
    if 'Both' in model_variant:
        lambda_ = 0.5
    
    # Configure attack model
    if 'A' in model_variant:
        model_class = MetaApprox
    else:
        model_class = Metattack
    
    # Evaluate across different perturbation rates
    for ptb_rate in ptb_rates:
        print(f'\n=== Testing perturbation rate: {ptb_rate*100:.1f}% ===')
        perturbations = int(ptb_rate * (adj.sum()//2))
        
        print('=== Setting up attack model ===')
        model = model_class(nfeat=features.shape[1], hidden_sizes=[hidden],
                        nnodes=adj.shape[0], nclass=nclass, dropout=0.5,
                        train_iters=0, attack_features=False, lambda_=lambda_, device=device)
        
        if device != 'cpu':
            model = model.to(device)
            
        # Generate adversarial adjacency matrix focusing on test nodes
        print(f'=== Perturbing graph with {perturbations} edge modifications ===')
        
        modified_adj = model(features, adj, labels, idx_train, idx_test, perturbations, ll_constraint=False)
        modified_adj = modified_adj.detach()
        
        runs = 3  # Reduced from 10 to 3 for quicker execution
        clean_acc = []
        attacked_acc = []
        
        print('=== Evaluating GCN performance ===')
        # Test the already trained GCN on both clean and perturbed graph
        for i in range(runs):
            # Reset the GCN for each run
            trained_gcn = train_gcn(adj)
            
            # Evaluate on clean test data
            clean_acc.append(evaluate(trained_gcn, adj, idx_test))
            
            # Evaluate on perturbed test data (evasion)
            attacked_acc.append(evaluate(trained_gcn, modified_adj, idx_test))
            
            print(f"Run {i+1}/{runs}: Clean acc = {clean_acc[-1]:.4f}, Attacked acc = {attacked_acc[-1]:.4f}")
        
        # Calculate effectiveness metrics
        clean_mean = np.mean(clean_acc)
        clean_std = np.std(clean_acc)
        attack_mean = np.mean(attacked_acc)
        attack_std = np.std(attacked_acc)
        acc_drop = clean_mean - attack_mean
        relative_drop = (acc_drop / clean_mean) * 100
        
        # Print summary statistics
        print(f"\n=== Attack Effectiveness Summary (Perturbation rate: {ptb_rate*100:.1f}%) ===")
        print(f"Clean accuracy: {clean_mean:.4f} ± {clean_std:.4f}")
        print(f"Attacked accuracy: {attack_mean:.4f} ± {attack_std:.4f}")
        print(f"Absolute accuracy drop: {acc_drop:.4f}")
        print(f"Relative accuracy drop: {relative_drop:.2f}%")
        print(f"Effectiveness ratio: {acc_drop/ptb_rate:.4f} (drop per perturbation unit)")
        
        # Assessment
        if acc_drop > 0.10:
            print("Attack assessment: Highly effective")
        elif acc_drop > 0.05:
            print("Attack assessment: Moderately effective")
        elif acc_drop > 0.02:
            print("Attack assessment: Slightly effective")
        else:
            print("Attack assessment: Minimally effective")
        
        # Store results
        results[ptb_rate] = {
            'modified_adj': modified_adj,
            'clean_acc': clean_acc,
            'attacked_acc': attacked_acc,
            'accuracy_drop': acc_drop,
            'relative_drop': relative_drop,
            'effectiveness_ratio': acc_drop/ptb_rate
        }
        
        # Free up memory
        del model
        torch.cuda.empty_cache()
        gc.collect()
    
    # Add overall results
    results['clean_adj'] = adj
    
    # Create and save comparative visualization
    plt.figure(figsize=(10, 6))
    
    ptb_values = list(ptb_rates)
    acc_drops = [results[ptb]['accuracy_drop'] for ptb in ptb_rates]
    rel_drops = [results[ptb]['relative_drop'] for ptb in ptb_rates]
    
    plt.subplot(1, 2, 1)
    plt.plot(ptb_values, acc_drops, 'o-', linewidth=2)
    plt.xlabel('Perturbation Rate')
    plt.ylabel('Absolute Accuracy Drop')
    plt.title('Impact of Perturbation Rate on Accuracy Drop')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(ptb_values, rel_drops, 'o-', linewidth=2, color='orange')
    plt.xlabel('Perturbation Rate')
    plt.ylabel('Relative Accuracy Drop (%)')
    plt.title('Impact of Perturbation Rate on Relative Accuracy Drop')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('perturbation_impact.png')
    plt.close()
    
    return results

In [ ]:
def clean_memory():
    """Clean GPU memory and garbage collect"""
    if device != 'cpu':
        torch.cuda.empty_cache()
    gc.collect()

In [9]:
if __name__ == '__main__':
    torch.cuda.empty_cache()
    results = run_attack()
    
    # Print comparative summary
    print("\n=== Comparative Analysis ===")
    print("Perturbation Rate | Accuracy Drop | Relative Drop | Effectiveness Ratio")
    print("-" * 65)
    for ptb_rate in ptb_rates:
        print(f"{ptb_rate*100:15.1f}% | {results[ptb_rate]['accuracy_drop']:12.4f} | {results[ptb_rate]['relative_drop']:12.2f}% | {results[ptb_rate]['effectiveness_ratio']:18.4f}")

=== Training GCN on original(clean) graph ===

=== Testing perturbation rate: 5.0% ===
=== Setting up attack model ===
=== Perturbing graph with 253 edge modifications ===
=== training surrogate model to predict unlabled data for self-training


Perturbing graph:   2%|█▎                                                               | 5/253 [00:00<00:15, 15.69it/s]

GCN loss on unlabled data: 1.9972083568572998
GCN acc on unlabled data: 0.08199195171026158
attack loss: 2.0009281635284424
GCN loss on unlabled data: 1.9898900985717773
GCN acc on unlabled data: 0.10010060362173039
attack loss: 1.9905608892440796
GCN loss on unlabled data: 1.9860377311706543
GCN acc on unlabled data: 0.11217303822937627
attack loss: 1.9758397340774536
GCN loss on unlabled data: 1.9910742044448853
GCN acc on unlabled data: 0.10663983903420524
attack loss: 1.9914789199829102
GCN loss on unlabled data: 2.010406970977783
GCN acc on unlabled data: 0.09758551307847084
attack loss: 2.0094587802886963
GCN loss on unlabled data: 1.9779176712036133
GCN acc on unlabled data: 0.11670020120724348
attack loss: 1.972975492477417


Perturbing graph:   5%|███▎                                                            | 13/253 [00:00<00:08, 27.55it/s]

GCN loss on unlabled data: 1.9916770458221436
GCN acc on unlabled data: 0.10814889336016098
attack loss: 1.9926518201828003
GCN loss on unlabled data: 1.9945136308670044
GCN acc on unlabled data: 0.07494969818913481
attack loss: 1.9986011981964111
GCN loss on unlabled data: 1.9673984050750732
GCN acc on unlabled data: 0.12273641851106641
attack loss: 1.9763991832733154
GCN loss on unlabled data: 1.9759411811828613
GCN acc on unlabled data: 0.09456740442655936
attack loss: 1.9710333347320557
GCN loss on unlabled data: 1.9973832368850708
GCN acc on unlabled data: 0.10663983903420524
attack loss: 2.005134344100952
GCN loss on unlabled data: 1.980126976966858
GCN acc on unlabled data: 0.14688128772635817
attack loss: 1.9745882749557495
GCN loss on unlabled data: 2.0121164321899414
GCN acc on unlabled data: 0.18611670020120727
attack loss: 2.024221420288086
GCN loss on unlabled data: 1.9586122035980225
GCN acc on unlabled data: 0.15392354124748492
attack loss: 1.9638410806655884


...

GCN loss on unlabled data: 2.0294253826141357
GCN acc on unlabled data: 0.07142857142857144
attack loss: 2.0263006687164307
GCN loss on unlabled data: 1.9362049102783203
GCN acc on unlabled data: 0.21327967806841047
attack loss: 1.941520094871521
GCN loss on unlabled data: 1.9776625633239746
GCN acc on unlabled data: 0.17454728370221329
attack loss: 1.9722075462341309
GCN loss on unlabled data: 2.0244405269622803
GCN acc on unlabled data: 0.13531187122736418
attack loss: 2.026524782180786
GCN loss on unlabled data: 1.9433159828186035
GCN acc on unlabled data: 0.18309859154929578
attack loss: 1.942708134651184
GCN loss on unlabled data: 1.9290882349014282
GCN acc on unlabled data: 0.19466800804828976
attack loss: 1.930700421333313
GCN loss on unlabled data: 2.0050570964813232
GCN acc on unlabled data: 0.10110663983903422
attack loss: 2.014282703399658
GCN loss on unlabled data: 1.8979237079620361
GCN acc on unlabled data: 0.2570422535211268
attack loss: 1.8969149589538574
GCN loss on un

Perturbing graph:  37%|███████████████████████▏                                       | 186/506 [00:04<00:08, 39.79it/s]

GCN loss on unlabled data: 1.911853313446045
GCN acc on unlabled data: 0.23943661971830987
attack loss: 1.9165573120117188
GCN loss on unlabled data: 1.960938572883606
GCN acc on unlabled data: 0.16096579476861167
attack loss: 1.9587007761001587
GCN loss on unlabled data: 1.9682071208953857
GCN acc on unlabled data: 0.10160965794768613
attack loss: 1.9638067483901978


Perturbing graph:  37%|███████████████████████▏                                       | 186/506 [00:05<00:09, 34.08it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 31.74 GiB total capacity; 31.28 GiB already allocated; 21.12 MiB free; 31.29 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF